## Predição de Sucesso de Startups  

Este notebook tem como objetivo explorar e modelar dados reais de startups para prever se uma empresa terá sucesso (ativa/adquirida) ou insucesso (fechada).

O projeto engloba análise exploratória dos dados, tratamento de valores ausentes, seleção de variáveis, processamento e aplicação de modelos de classificação binária.

## 1. Importação de Bibliotecas

Importação das bibliotecas necessárias para manipulação de dados (Pandas, NumPy), visualização (Matplotlib, Seaborn), modelagem (Scikit-Learn) e ajuste de hiperparâmetros (RandomizedSearchCV) e métricas para avaliação

In [9]:
# Importação de bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score

### 2. Carregamento de dados

Os datasets de treino e teste são carregados em DataFrames pandas a partir de arquivos CSV. Exibe-se a dimensão, primeiras linhas, informações gerais, estatísticas descritivas e a incidência de valores ausentes para conhecer melhor os dados que se é trabalhado.

In [10]:
# Carregamento do dataset principal
train_df = pd.read_csv("../data/train.csv")
test_df = pd.read_csv("../data/test.csv")

print("Dados de treino carregados com sucesso. Shape: {}".format(train_df.shape))
print("Dados de teste carregados com sucesso. Shape: {}".format(test_df.shape))

print("Primeiras 5 linhas dos dados de treino:")
print(train_df.head(5))

Dados de treino carregados com sucesso. Shape: (646, 33)
Dados de teste carregados com sucesso. Shape: (277, 32)
Primeiras 5 linhas dos dados de treino:
    id  age_first_funding_year  age_last_funding_year  \
0  719                   10.42                  13.09   
1  429                    3.79                   3.79   
2  178                    0.71                   2.28   
3  197                    3.00                   5.00   
4  444                    0.66                   5.88   

   age_first_milestone_year  age_last_milestone_year  relationships  \
0                      8.98                    12.72              4   
1                       NaN                      NaN             21   
2                      1.95                     2.28              5   
3                      9.62                    10.39             16   
4                      6.21                     8.61             29   

   funding_rounds  funding_total_usd  milestones  is_CA  ...  is_consulting  

In [11]:
print("Informações gerais sobre os treinos:")
train_df.info()

print("Estatísticas descritivas dos treinos:")
print(train_df.describe())

print("Valores ausentes nos treinos:")
print(train_df.isnull().sum()[train_df.isnull().sum() > 0])

print("Distribuição da variável alvo (labels):")
print(train_df["labels"].value_counts())
print(train_df["labels"].value_counts(normalize=True) * 100)

Informações gerais sobre os treinos:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 646 entries, 0 to 645
Data columns (total 33 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        646 non-null    int64  
 1   age_first_funding_year    611 non-null    float64
 2   age_last_funding_year     637 non-null    float64
 3   age_first_milestone_year  508 non-null    float64
 4   age_last_milestone_year   535 non-null    float64
 5   relationships             646 non-null    int64  
 6   funding_rounds            646 non-null    int64  
 7   funding_total_usd         646 non-null    int64  
 8   milestones                646 non-null    int64  
 9   is_CA                     646 non-null    int64  
 10  is_NY                     646 non-null    int64  
 11  is_MA                     646 non-null    int64  
 12  is_TX                     646 non-null    int64  
 13  is_otherstate             64

## 3. Tratamento de Valores Ausentes

As variáveis numéricas que apresentam dados faltantes (`NaN`), especialmente relacionadas a anos de eventos financeiros e de marcos importantes, são tratadas pela imputação da mediana da respectiva coluna, tanto no treino quanto no teste dos conjuntos de dados.

Com isso é mitigado o impacto de valores faltantes sem enviesar o modelo.

In [12]:
# 3. Tratando dados ausentes
cols_to_impute_median = [
    'age_first_funding_year', 'age_last_funding_year',
    'age_first_milestone_year', 'age_last_milestone_year',
    'funding_total_usd'
]

for col in cols_to_impute_median:
    median_val = train_df[col].median()
    train_df[col].fillna(median_val, inplace=True)
    test_df[col].fillna(median_val, inplace=True)

/tmp/ipykernel_48066/2381036111.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_df[col].fillna(median_val, inplace=True)
/tmp/ipykernel_48066/2381036111.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', tr

## 4. Codificação de Variáveis Categóricas

A variável categórica `category_code` é transformada em variáveis dummy (one-hot encoding), convertendo os setores em colunas binárias para o modelo.  
Depois, garante-se o alinhamento entre conjunto de treino e teste, adicionando colunas faltantes com zeros para evitar inconsistências.

In [13]:
# Aplicando one hot encoding para category_code
categorical_col = ["category_code"]
train_df = pd.get_dummies(train_df, columns=categorical_col, drop_first=True)
test_df = pd.get_dummies(test_df, columns=categorical_col, drop_first=True)

missing_cols_in_test = set(train_df.columns) - set(test_df.columns)
missing_cols_in_train = set(test_df.columns) - set(train_df.columns)

for col in missing_cols_in_test:
    if col != 'labels':
        test_df[col] = 0
for col in missing_cols_in_train:
    if col != 'labels':
        train_df[col] = 0


## 5. Separação de Features e Target e Padronização

Define-se o conjunto de variáveis explicativas (features) e a variável alvo (target).   
Selecionam-se as colunas numéricas para padronização usando o StandardScaler, equilibrando a escala dos dados e trazendo melhora para desempenho do modelo.

In [14]:
# 4. Preparação de features e target
X_train_full = train_df.drop("labels", axis=1)
y_train_full = train_df["labels"]
X_test = test_df.copy()

numerical_cols = [
    col for col in X_train_full.columns
    if X_train_full[col].dtype in ["int64", "float64"]
    and not col.startswith("is_")
    and not col.startswith("category_code_")
]


## 6. Divisão em Treino e Validação

Divide-se o conjunto de treino original em treino e validação, utilizando uma amostra estratificada para preservar a proporção das classes.  
Assim, possibilita avaliação imparcial do modelo antes de validar no conjunto de teste.

In [15]:
# Divisão treino/validação estratificada
X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full,
    test_size=0.2,
    stratify=y_train_full,
    random_state=42)

In [16]:
# Aplicando padronização
scaler = StandardScaler()

X_train_scaled = X_train.copy()
X_val_scaled = X_val.copy()
X_test_scaled = X_test.copy()

X_train_scaled[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
X_val_scaled[numerical_cols] = scaler.transform(X_val[numerical_cols])
X_test_scaled[numerical_cols] = scaler.transform(X_test[numerical_cols])


## 7. Ajuste de Hiperparâmetros com RandomizedSearchCV

Utiliza-se RandomizedSearchCV para ajustar o hiperparâmetro de regularização C da regressão logística.  
Busca aleatória e validação cruzada (5-fold) são aplicadas para encontrar valores que maximizem a acurácia e previnam overfitting.

In [18]:
# Seleção de features baseada em importância com LogisticRegression
selector = SelectFromModel(
    LogisticRegression(class_weight='balanced', solver='liblinear', max_iter=500, random_state=42),
    threshold='mean'
)

# Ajusta o seletor com treino
selector.fit(X_train_scaled, y_train)

# Transforma treino e validação normalmente
X_train_sel = selector.transform(X_train_scaled)
X_val_sel = selector.transform(X_val_scaled)

# Para o conjunto de teste, alinhe as colunas de X_test_scaled com as do X_train_scaled
X_test_aligned = X_test_scaled[X_train_scaled.columns]

# Depois transforme com selector
X_test_sel = selector.transform(X_test_aligned)

print(f'Número de features antes da seleção: {X_train_scaled.shape[1]}')
print(f'Número de features após a seleção: {X_train_sel.shape[1]}')

Número de features antes da seleção: 65
Número de features após a seleção: 24


In [19]:
# Definição para os hiperparâmetros C 
rf = RandomForestClassifier(class_weight='balanced', random_state=42)

param_dist = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

random_search_rf = RandomizedSearchCV(
    rf,
    param_dist,
    n_iter=30,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    random_state=42,
    verbose=1
)

random_search_rf.fit(X_train_sel, y_train)

print(f'Melhor estimador Random Forest: {random_search_rf.best_params_}')

Fitting 5 folds for each of 30 candidates, totalling 150 fits
Melhor estimador Random Forest: {'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_depth': 20, 'bootstrap': True}


## 8. Avaliação do Modelo

O melhor modelo encontrado é avaliado no conjunto de validação com métricas completas, incluindo relatório de classificação e acurácia.

In [20]:
# Avaliação do modelo no conjunto de validação
y_val_pred = random_search_rf.predict(X_val_sel)

print("Relatório de classificação no conjunto de validação:")
print(classification_report(y_val, y_val_pred))
print("Acurácia no conjunto de validação: {:.2f}".format(accuracy_score(y_val, y_val_pred)))

Relatório de classificação no conjunto de validação:
              precision    recall  f1-score   support

           0       0.67      0.70      0.68        46
           1       0.83      0.81      0.82        84

    accuracy                           0.77       130
   macro avg       0.75      0.75      0.75       130
weighted avg       0.77      0.77      0.77       130

Acurácia no conjunto de validação: 0.77


## 9. Treinamento Final e Previsão no Conjunto de Teste

Treina-se o modelo final com todos os dados de treino utilizando o melhor hiperparâmetro.  
Realizam-se as previsões no conjunto de teste, garantindo que as colunas estejam alinhadas com o conjunto de treino, para evitar erros.

In [21]:
# Treinamento final com tudo e predição no teste
best_model = random_search_rf.best_estimator_
best_model.fit(selector.transform(X_train_full), y_train_full)

submission_preds = best_model.predict(X_test_sel)

submission_df = pd.DataFrame({
    'id': test_df['id'],
    'labels': submission_preds
})

submission_df.to_csv('../data/submission.csv', index=False)